# IPython Logbook Manager

This IPython notebook can be used to manage the Logbook via a collection of bash scripts that handle the listing, creating, and backing up of the logbook entries. Each subsection title corresponds to the function performed by the bash script contained within. The subsection is divided into two parts: the first contains variables that must set by the user for the bash script; the second contains the script itself. The user should execute an action only by *individual cell executations* of only those cells that pertain to the action he/she wishes to enact. 

Two layers of protection are implemented to prevent accidental executation of a script. The first is that - by default - the bash script cells are marked 'read only' using the IPython-notebook-extension 'read-only.js': the user must click the little 'lock' icon at the upper-right. The second is that the user must set a 'Script_Execute' flag to 'Yes'. Despite these protections, it is strongly recommended that the user ** not execute a *Cell $\rightarrow$ Run All* command** while working within this notebook!

In [5]:
import ConfigParser
CP = ConfigParser.ConfigParser()
CP.read("../.config")
head = CP.get('IPyLogbook-Config','head')
url = CP.get('IPyLogbook-Config','url')
port = CP.get('IPyLogbook-Config','ssh-port')

headLink="[Logbook HEAD]("+url+":"+port+"/tree)"
extensionsLink="[Logbook Extensions]("+url+":"+port+"/notebooks/IPyLogbook/mgmt/IPyLogbookExtensions.ipynb)"
indexLink="[Logbook Index]("+url+":"+port+"/notebooks/IPyLogbook/IPyLogbookIndex.ipynb)"
usersguideLink="[Logbook User Guide]("+url+":"+port+"/notebooks/IPyLogbook/doc/IPyLogbookUsersGuide.ipynb)"

## Create Logbook Entry

When creating a new logbook entry, don't forget to update the {{indexLink}} accordingly!  
To create a logbook entry, please set the following variables appropriately:

In [198]:
CreateScript_Dir=head+"/experiment/20140101" # Abs. path to directory where Logbook entry will be created
CreateScript_Name="20140101" # Name of the Logbook entry (will be used in name if notebook file)
CreateScript_Execute="No" # "Yes" = run script; "No" = do not run script
CreateScript_Overwrite="No"  # "Yes" = overwrite preexising log entry; "No" = do NOT overwrite preexisting log entry

In [199]:
#-------------------- The user should not need to set anything below this line --------------------#

In [200]:
%%bash -s "$CreateScript_Dir" "$CreateScript_Name" "$CreateScript_Execute" "$CreateScript_Overwrite" "$head"

if [ "$#" -ne 5 ]; then
    echo -e "\nError: This script requires four arguments that should be set by the user!"
    echo -e   "arg1 : Absolute path to where new Logbook entry will be created
    echo -e   "arg2 : Name of the new Logbook entry
    echo -e   "arg3 : Yes/No to run the script"
    echo -e   "arg4 : Yes/No to overwrite an existing Logbook entry with the name specified"
    echo -e   "arg5 : Full path to the HEAD directory\n"
    exit
fi

# Ensure that the user has intentionally flagged this script to run
if [ "$3" == "No" ]; then
    echo -e "\nThis script is not flagged for execution. Set 'CreateScript_Execute' flag to 'Yes' to execute"
    exit
fi

# Set full path to the directory containing the new entry
EntryDir=$1

# Set the new entry's file name
EntryName="IPyLogbookEntry-"$2".ipynb"

# Set the full path to the new entry file
Entry=$EntryDir/$EntryName

# If the directory does NOT exist then create it
if [ ! -d $EntryDir ]; then
    mkdir -p $EntryDir

# If the directory DOES exist then...
else
    # ... overwrite the preexisting Logbook entry if the user has granted permission to do so
    if [ "$4" == "No" ]; then
        echo -e "\nWARNING : The Logbook entry '$Entry' already exists!"
        echo -e   "          You may set the above 'CreateScript_Overwrite' parameter to 'Yes' to overwrite this entry,"
        echo -e   "          but you should exercise extreme caution when using this option!\n"
        exit
    fi
fi

# Set the Logbook entry template to be copied to the new Logbook entry and copy it
EntryTemplate="IPyLogbookEntryTemplate.ipynb"
cp $EntryTemplate $Entry

# Place a symbolic link to the IPyLogbook config file so that the new Logbook entry will have access to it
if [ "$4" == "Yes" ]; then
    rm $EntryDir/.config -f
fi
    
ln -s $5/IPyLogbook/.config $EntryDir/.config

echo -e "\nA new Logbook entry was successfully created at:\n"
echo -e "    $Entry\n"


This script is not flagged for execution. Set 'CreateScript_Execute' flag to 'Yes' to execute


## List Logbook Entries

To list the available logbook entries, please set the following variables appropriately:

In [201]:
ListScript_Execute="No" # "Yes" = run script; "No" = do not run script

In [202]:
#-------------------- The user should not need to set anything below this line --------------------#

In [203]:
%%bash -s "$ListScript_Execute" "$head"

if [ "$#" -ne 2 ]; then
    echo -e "\nError: This script requires four arguments that should be set by the user!"
    echo -e   "arg1 : Yes/No to run the script"
    echo -e   "arg2 : Full path to the HEAD directory\n"
    exit
fi

# Ensure that the user has intentionally flagged this script to run
if [ "$1" == "No" ]; then
    echo -e "\nThis script is not flagged for execution. Set 'ListScript_Execute' flag to 'Yes' to execute".
    exit
fi

find $2 -name "IPyLogbookEntry-*.ipynb"


This script is not flagged for execution. Set 'ListScript_Execute' flag to 'Yes' to execute.


## Backup Logbook

To backup the logbook to the directory of your choice, please set the following variables appropriately:

In [207]:
BackupScript_Directory="/home/hartwig/logbook/backup" # Full path to backup directory
BackupScript_Execute="No" # "Yes" = run script; "No" = do not run script
BackupScript_Overwrite="No" # "Yes" = overwrite preexising backup; "No" = do NOT overwrite preexisting backup

In [208]:
#-------------------- The user should not need to set anything below this line --------------------#

In [209]:
%%bash -s  "$BackupScript_Directory" "$BackupScript_Execute" "$BackupScript_Overwrite" "$head"

# Ensure correct number or arguments are passed; provide helpful output
if [ "$#" -ne 4 ]; then
    echo -e "\nError: This script requires four arguments that should be set by the user!"
    echo -e   "arg1 : Absolute path to where the Logbook will be backed up"
    echo -e   "arg2 : Yes/No to run the script"
    echo -e   "arg3 : Yes/No to overwrite an existing Logbook entry with the name specified"
    echo -e   "arg4 : Full path to the HEAD directory\n"
     exit
fi

# Put cmd line args into reasonably named variables
Directory=$1
Execute=$2
Overwrite=$3
Head=$4

# Ensure that the user has intentionally flagged this script to run
if [ "$Execute" == "No" ]; then
    echo -e "\nThis script is not flagged for execution. Set 'ListScript_Execute' flag to 'Yes' to execute"
    exit
fi

# Check to see if a directory already exists where the backup will be made
if [ -d $Directory ]; then
    # Prevent overwriting the directory; provide advice to overwrite
     if [ "$Overwrite" == "No" ]; then
        echo -e "\nA backup of this logbook already exists at $1! Set 'BackupScript_Overwrite' to 'Yes' to overwrite."
        echo -e   "Please exercise CAUTION when using this option!\n"
        exit
        
    # Remove and recreate the directory
    elif [ "$Overwrite" == "Yes" ]; then
        chmod 755 $Directory
        rm $Directory -rf
        mkdir $Directory
    fi  
else
    mkdir -p $Directory
fi

# Change to the IPyLogbook HEAD directory and copy all of the IPyLogbook entries to
# the specified directory being sure to preserve the directory structure!
cd $Head
EntryList=$(find . -name 'IPyLogbookEntry-*.ipynb' | grep -v '.ipynb_checkpoints')
for Entry in $EntryList; do
    Entry=${Entry#.\/}
    cp --parents $Entry $Directory
done

cat > $Directory/README.txt << EOL
                              *************
                              ** WARNING **
                              *************
                              
This directory contains a backup of an IPyLogbook! Please treat with respect!

EOL
Date=$(date)

echo "name: README.txt" >> $Directory/README.txt
echo "date: "$Date >> $Directory/README.txt

echo -e "\nA backup of this IPyLogbook was created at $Directory that includes the following files:"

for Entry in $EntryList; do
    Entry=${Entry#.\/}
    echo "  "$Entry  
done


This script is not flagged for execution. Set 'ListScript_Execute' flag to 'Yes' to execute


---

## <center> Logbook Navigation</center>
<center>{{headLink}} --- {{extensionsLink}} --- {{indexLink}} --- {{usersguideLink}}</center>